In [2]:


import requests 
import pandas as pd
import numpy as np
from pprint import pprint
import json
import time
import ast
import datetime
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt


In [3]:
token = 'c236a984c236a984c236a9846bc1258e29cc236c236a984a62e08a4142889748be3012f' 
url = 'https://api.vk.com/method/groups.getMembers' 

count = 1000 
offset = 0  
user_ids = []  
while offset < 5000: 
    
    params = {'group_id': 'vaflya_house', 
              'v': 5.95,    
              'count': count, 
              'offset': offset, 
              'access_token': token,
              'extended': 1,
              'fields': 'first_name, last_name, city, country, about, activities, bdate, connections, contacts, occupation, last_seen, interests, sex'
              } 
    
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    offset += count 
    print('Ожидаю 0.5 секунды...') 
    time.sleep(0.5) 
print('Цикл завершен, offset =',offset) 

Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Цикл завершен, offset = 5000


In [4]:
user_df = pd.DataFrame(user_ids)

def extract_city_name(city_dict):
    if isinstance(city_dict, dict):
        return city_dict['title']
    return None

def extract_work_name(work_dict):
    if isinstance(work_dict, dict):
        return work_dict['name']
    return None

def extract_plathorm_name(platform_dict):
    if isinstance(platform_dict, dict):
        return platform_dict['platform']
    return None

user_df['city'] = user_df['city'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
user_df['country'] = user_df['country'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
user_df['occupation'] = user_df['occupation'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
user_df['last_seen'] = user_df['last_seen'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


user_df['city'] = user_df['city'].apply(extract_city_name)
user_df['country'] = user_df['country'].apply(extract_city_name)
user_df['occupation'] = user_df['occupation'].apply(extract_work_name)
user_df['platform'] = user_df['last_seen'].apply(extract_plathorm_name)

user_df['platform'] = user_df['platform'].fillna(0) 


user_df['platform_text'] = user_df['platform'].map({
    1.0: 'мобильная версия',
    2.0: 'приложение для iPhone',
    3.0: 'приложение для iPad',
    4.0: 'приложение для Android',
    5.0: 'приложение для Windows Phone',
    6.0: 'приложение для Windows 10',
    7.0: 'полная версия сайта',
    0.0: 'не известно'
    })


def convert_to_datetime(date_str):
    try:
        date_obj = parse(date_str, dayfirst=True)
        return date_obj.strftime('%Y-%m-%d')  
    except ValueError:
        return None  

user_df['bdate'] = user_df['bdate'].astype(str)
user_df['birthdate'] = user_df['bdate'].apply(convert_to_datetime)
user_df['birthdate'] = pd.to_datetime(user_df['birthdate'], errors='coerce')


today = datetime.date.today()

user_df['age'] = user_df['birthdate'].apply(lambda x: relativedelta(today, x).years if pd.notnull(x) else None)
user_df['age'] = user_df['age'].astype('Int64')

user_df['sex_text'] = user_df['sex'].map({
    1.0: 'Девушка',
    2.0: 'Мужчина',
    3.0: 'Не известно',
    })

user_df['age'] = user_df['age'].replace(0, np.nan)
user_df['age'] = user_df['age'].replace(range(80, 122), np.nan)
user_df.platform_text.value_counts()

приложение для Android    1882
приложение для iPhone     1702
полная версия сайта        326
мобильная версия           270
не известно                 50
приложение для iPad          1
Name: platform_text, dtype: int64

In [5]:
user_df_final = user_df[['id', 'city', 'country', 'skype', 'mobile_phone', 'interests', 'about', 'activities', 'first_name', 'last_name' , 'platform_text', 'age', 'sex_text']]

# user_df_final.to_csv('вафли_house_vk.csv')
# user_df_final.to_csv('vaflya_house_vk_v2.csv')